# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
cities_weather_data = "cities_dataframe.csv"


cities_weather_data = pd.read_csv(cities_weather_data)
cities_weather_data

,City Name,Lattitude,Longitude,temperature,Humidity,Cloudiness,Windspeed
0,Bluff,-46.60,168.33,37.99,96,100,1.99
1,Saint-Louis,16.33,-15.00,96.80,47,40,4.70
2,Port Elizabeth,-33.92,25.57,53.60,93,90,3.36
3,Mitú,1.20,-70.17,76.66,89,91,3.09
4,Kaitangata,-46.28,169.85,45.00,91,84,1.99
...,...,...,...,...,...,...,...
556,Yārāda,17.65,83.27,82.40,83,75,11.41
557,Vuktyl,63.86,57.31,64.90,78,72,6.02
558,Mahon,39.89,4.27,80.60,61,20,5.82
559,Port Blair,11.67,92.75,82.17,84,100,14.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [17]:
locations = cities_weather_data[["Lattitude", "Longitude"]].astype(float)
humidity = cities_weather_data["Humidity"].astype(float)
fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius = 8)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
vacation_destinations = cities_weather_data[cities_weather_data['temperature']>=70] 
vacation_destinations = cities_weather_data[cities_weather_data['temperature']<=80]
vacation_destinations = cities_weather_data[cities_weather_data['Humidity']<=70]
vacation_destinations = cities_weather_data[cities_weather_data['Windspeed']<=8]
vacation_destinations = cities_weather_data[cities_weather_data['Cloudiness']<=5]

vacation_destinations

,City Name,Lattitude,Longitude,temperature,Humidity,Cloudiness,Windspeed
8,Busselton,-33.65,115.33,48.42,80,2,7.90
12,Chokurdakh,70.63,147.92,48.13,72,0,7.47
20,Eirunepé,-6.66,-69.87,96.08,28,0,3.20
21,Saldanha,-33.01,17.94,53.60,50,0,10.29
23,Qaanaaq,77.48,-69.36,36.18,74,0,3.51
...,...,...,...,...,...,...,...
541,Nokaneng,-19.67,22.27,66.36,26,0,2.68
542,Punta Gorda,26.93,-82.05,88.90,66,1,5.82
545,Miles City,46.41,-105.84,78.80,39,1,4.70
546,Mount Isa,-20.73,139.50,55.40,50,0,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df=vacation_destinations
hotel_df.head()

,City Name,Lattitude,Longitude,temperature,Humidity,Cloudiness,Windspeed
8,Busselton,-33.65,115.33,48.42,80,2,7.90
12,Chokurdakh,70.63,147.92,48.13,72,0,7.47
20,Eirunepé,-6.66,-69.87,96.08,28,0,3.20
21,Saldanha,-33.01,17.94,53.60,50,0,10.29
23,Qaanaaq,77.48,-69.36,36.18,74,0,3.51


In [21]:
params = {
    "key": g_key,
    "radius" : 5000
}

for index, row in hotel_df.iterrows():
    

    # base_url = "http://api.openweathermap.org/data/2.5/onecall/timemachine"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    latitude = row['Lattitude']
    longitude = row['Longitude']
    
    params["location"] = f"{latitude},{longitude}"

    hotel = requests.get(base_url,params=params)

    hotel = hotel.json()
    
    try:
        hotel_df.loc[index, "Hotel_Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info error ----- skip.")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing info error ----- skip.
Missing info error ----- skip.


In [26]:
hotel_df

,City Name,Lattitude,Longitude,temperature,Humidity,Cloudiness,Windspeed,Hotel_Name
8,Busselton,-33.65,115.33,48.42,80,2,7.90,Busselton
12,Chokurdakh,70.63,147.92,48.13,72,0,7.47,Chokurdakh
20,Eirunepé,-6.66,-69.87,96.08,28,0,3.20,Eirunepé
21,Saldanha,-33.01,17.94,53.60,50,0,10.29,Saldanha
23,Qaanaaq,77.48,-69.36,36.18,74,0,3.51,Qaanaaq
...,...,...,...,...,...,...,...,...
541,Nokaneng,-19.67,22.27,66.36,26,0,2.68,NaN
542,Punta Gorda,26.93,-82.05,88.90,66,1,5.82,Port Charlotte
545,Miles City,46.41,-105.84,78.80,39,1,4.70,Miles City
546,Mount Isa,-20.73,139.50,55.40,50,0,8.05,Mount Isa


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Busselton</dd>\n<dt>City</dt><dd>Busselton</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Chokurdakh</dd>\n<dt>City</dt><dd>Chokurdakh</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Eirunepé</dd>\n<dt>City</dt><dd>Eirunepé</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Saldanha</dd>\n<dt>City</dt><dd>Saldanha</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Qaanaaq</dd>\n<dt>City</dt><dd>Qaanaaq</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Severnoye</dd>\n<dt>City</dt><dd>Severnoye</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Sovetsky</dd>\n<dt>City</dt><dd>Sovetskiy</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Moosomin</dd>\n<dt>City</dt><dd>Moosomin</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Luderitz</dd>\n<dt>City</dt><dd>Lüderitz</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Voorheesville</dd>\n<dt>City</dt><dd>Albany</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Matamoros</dd>\n<dt>City</dt><dd>Matamoros</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Geraldton</dd>\n<dt>City</dt><dd>Gera

In [25]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))